In [1]:
# Copyright 2018 Anar Amirli
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train/Test Set Construction
In this document, we construct training and test sets from already computed feature sets. The sets are computed according to the layout:

data<br>
├── train_features<br>
├── test_features<br>

In [2]:
from os import listdir
from os.path import join
import pickle
import numpy as np
import pandas as pd
import multiprocessing
from collections import Counter
from random import shuffle
import json
import re

# our common utility functions that are used in different notebooks
from utils import *

pd.set_option('compute.use_bottleneck', True)
pd.set_option('compute.use_numexpr', True)

In [3]:
def describe_dataset(dataset):
    print('Shape: {}\n'.format(dataset.shape))
    
    _, col = dataset.shape
        
    heading = 'Pitch index counts'
    print(heading + '\n' + '-'*len(heading))
    for key, val in sorted(Counter(dataset.values[:, col-1]).items()):
        print('{}\t: {}'.format(int(key), val))

## Pitch IDS

In [4]:
if __name__ == '__main__':

    match_id = 60565
    try:
        features_df = pd.read_csv('../data/general/feature-set/match{}_features.csv'.format(match_id))
    except FileNotFoundError:
        pass

    ball_idxs = np.sort(features_df['pitch_index'].unique())
    print('Ball  index: {}'.format(ball_idxs))

    print('Size: {}'.format(len(ball_idxs)))

Ball  index: [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.]
Size: 27


Null index(0) represents all the event categories, apart from the ones we are interested in, that occur when the game stops.

## Dataset Construction

In this section, we construct a combined shuffled dataset for train and set sets, coming from feature data.


### Parameters

When obtaining feature data for a given index, set we should reduce the number of frames of the null ball events in our data set. But cosidering the fact that they are most common, we should keep their numbers close to the number of other most occurred ball location indexes.

* $-$: No-labaled positions intervals would lead to greater label noise.

### Construction
Here we construct the combined dataset from all match data we have in the given data directories.

In [5]:
    # file based constants
    features_dir     = '../data/general/feature-set/'
    features_regex   = re.compile(r'match\d+_features.csv')

We speed up the computation by using all the CPU cores via multiprocessing module.

In [6]:
    pool = multiprocessing.Pool()
    df = pd.DataFrame()

    
    features_csv_files = [f for f in listdir(features_dir) if features_regex.match(f)]
    shuffle(features_csv_files)
    print('All matches: {}'.format(len(features_csv_files)))
    features_train_csv = features_csv_files[0:259]
    features_test_csv  = features_csv_files[259:len(features_csv_files)]
    print('Train matches: {}'.format(len(features_train_csv)))
    print('Test matches: {}'.format(len(features_test_csv)))
    
    
    df_train = pd.concat(pool.map(construct_train_set, features_train_csv))
    df_test = pd.concat(pool.map(construct_test_set, features_test_csv))


    print('Train-set shape: {}'.format(df_train.shape))
    print('Test-set shape: {}'.format(df_test.shape))

All matches: 299
Train matches: 259
Test matches: 40
Train-set shape: (720963, 250)
Test-set shape: (188376, 250)


In [7]:
    # shuffling data-sets agin
    df_train = df_train.sample(frac=1).reset_index(drop=True)
    df_test = df_test.sample(frac=1).reset_index(drop=True)

### Train Set

In [8]:
    describe_dataset(df_train)
    display(df_train.head())

Shape: (720963, 250)

Pitch index counts
------------------
0	: 36260
1	: 17915
2	: 22567
3	: 16459
4	: 26775
5	: 19711
6	: 20625
7	: 18784
8	: 27588
9	: 27742
10	: 26656
11	: 33906
12	: 34807
13	: 33584
14	: 33723
15	: 34690
16	: 34340
17	: 28625
18	: 28117
19	: 29744
20	: 21538
21	: 22771
22	: 23706
23	: 19970
24	: 32108
25	: 22457
26	: 25795


,# refX,refY,refSpeed,refDirectX,refDirectY,homeMaxX,homeMinX,homeMaxY,homeMinY,homeAvgX,...,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardSlowDensity,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardHirDensity,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
0,66.500000,29.656250,3.019531,1.0,-1.0,72.9375,42.84375,64.18750,3.839844,60.56250,...,0.00000,0.000000,0.0,43.5625,45.375,1.0,0.00000,0.00000,0.000000,15.0
1,80.875000,24.812500,3.289062,1.0,1.0,93.9375,50.25000,51.40625,2.910156,71.50000,...,49.59375,37.343750,1.0,0.0000,0.000,0.0,0.00000,0.00000,0.000000,20.0
2,51.218750,37.437500,1.820312,1.0,1.0,79.5000,40.96875,63.59375,12.796875,59.53125,...,0.00000,0.000000,0.0,0.0000,0.000,0.0,39.96875,23.59375,3.480469,22.0
3,79.125000,13.640625,0.529785,1.0,1.0,88.5000,57.28125,32.46875,4.730469,73.18750,...,67.75000,7.929688,1.0,0.0000,0.000,0.0,0.00000,0.00000,0.000000,23.0
4,19.046875,54.312500,0.919922,1.0,1.0,49.0000,8.93750,64.50000,23.187500,24.37500,...,0.00000,0.000000,0.0,0.0000,0.000,0.0,0.00000,0.00000,0.000000,9.0


### Test Set

In [9]:
    describe_dataset(df_test)
    display(df_test.head())

Shape: (188376, 250)

Pitch index counts
------------------
0	: 84233
1	: 2867
2	: 3220
3	: 2832
4	: 4611
5	: 3385
6	: 3113
7	: 3311
8	: 4537
9	: 4335
10	: 4263
11	: 5188
12	: 5211
13	: 5127
14	: 5179
15	: 5419
16	: 5050
17	: 4257
18	: 4317
19	: 4118
20	: 3123
21	: 3240
22	: 3080
23	: 2883
24	: 4546
25	: 2925
26	: 4006


,# refX,refY,refSpeed,refDirectX,refDirectY,homeMaxX,homeMinX,homeMaxY,homeMinY,homeAvgX,...,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardSlowDensity,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardHirDensity,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
0,61.187500,24.140625,4.179688,-1.0,1.0,72.25000,47.375000,48.25000,6.859375,61.50000,...,0.000000,0.0000,0.000000,43.531250,32.250000,1.000000,39.375,18.671875,3.210938,17.0
1,17.484375,52.625000,1.299805,1.0,0.0,39.34375,3.669922,63.68750,25.265625,20.62500,...,23.296875,62.6875,0.231445,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0
2,57.718750,27.406250,2.720703,-1.0,1.0,77.68750,45.218750,54.96875,10.273438,57.43750,...,0.000000,0.0000,0.000000,47.531250,25.828125,0.865723,0.000,0.000000,0.000000,18.0
3,90.437500,10.296875,1.860352,-1.0,1.0,100.37500,54.843750,43.62500,2.929688,78.37500,...,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,71.750,24.984375,4.019531,0.0
4,39.750000,26.406250,1.610352,1.0,1.0,58.06250,0.000000,51.50000,0.000000,39.65625,...,0.000000,0.0000,0.000000,22.390625,32.343750,1.000000,0.000,0.000000,0.000000,0.0


## Exporting

In [10]:
    df_train.to_csv('../data/general/{dataset}/all_{dataset}.csv'.format(dataset='train'), index=False)
    df_test.to_csv('../data/general/{dataset}/all_{dataset}.csv'.format(dataset='test'), index=False)

In [15]:
    df_test[1:10]

,# refX,refY,refSpeed,refDirectX,refDirectY,homeMaxX,homeMinX,homeMaxY,homeMinY,homeAvgX,...,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardSlowDensity,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardHirDensity,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
1,17.484375,52.625000,1.299805,1.0,0.0,39.343750,3.669922,63.68750,25.265625,20.625000,...,23.296875,62.68750,0.231445,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,57.718750,27.406250,2.720703,-1.0,1.0,77.687500,45.218750,54.96875,10.273438,57.437500,...,0.000000,0.00000,0.000000,47.531250,25.828125,0.865723,0.000000,0.000000,0.000000,18.0
3,90.437500,10.296875,1.860352,-1.0,1.0,100.375000,54.843750,43.62500,2.929688,78.375000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,71.750000,24.984375,4.019531,0.0
4,39.750000,26.406250,1.610352,1.0,1.0,58.062500,0.000000,51.50000,0.000000,39.656250,...,0.000000,0.00000,0.000000,22.390625,32.343750,1.000000,0.000000,0.000000,0.000000,0.0
5,35.281250,47.125000,1.549805,-1.0,-1.0,51.406250,18.953125,51.84375,19.187500,31.093750,...,0.000000,0.00000,0.000000,18.125000,27.562500,1.000000,0.000000,0.000000,0.000000,7.0
6,63.468750,24.500000,1.490234,-1.0,1.0,70.437500,38.218750,45.00000,19.203125,53.062500,...,36.875000,24.28125,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,27.953125,34.312500,0.399902,-1.0,-1.0,22.765625,0.000000,40.09375,0.000000,14.570312,...,0.000000,0.00000,0.000000,23.796875,28.906250,1.000000,12.117188,43.968750,3.279297,0.0
8,57.281250,24.937500,4.218750,1.0,1.0,66.187500,29.953125,54.65625,2.990234,48.593750,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,51.312500,47.125000,7.738281,0.0
9,28.187500,36.843750,3.390625,1.0,-1.0,44.937500,0.000000,46.46875,0.000000,24.437500,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,38.531250,34.062500,4.921875,0.0
